# Importación de Librerías.

In [5]:
import pandas as pd
import numpy as np


%matplotlib inline


import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, make_scorer
from catboost import CatBoostClassifier

# Lectura de Datos.

In [6]:
train_values = pd.read_csv('train_values.csv', index_col='building_id')
train_labels = pd.read_csv('train_labels.csv', index_col='building_id')

test_values = pd.read_csv('test_values.csv', index_col='building_id')
submission_format = pd.read_csv('submission_format.csv', index_col='building_id')

# Creación de Scorer.

In [7]:
f1 = make_scorer(f1_score , average='micro')

# Feature Engineering

In [8]:
#Geo Level Id 1. 

train_values.geo_level_1_id.describe() 

train_values['geo_level_1_id_less_than_10'] = [1 if (x <= 10) else 0 for x in train_values['geo_level_1_id']]

train_values['geo_level_1_id_10_20'] = [1 if ((x <= 20) & (x > 10))  else 0 \
                                        for x in train_values['geo_level_1_id']]

train_values['geo_level_1_id_higher_than_20'] = [1 if (x > 20) else 0 for x in train_values['geo_level_1_id']]


#Hacemos lo mismo para el set de Test. 


test_values['geo_level_1_id_less_than_10'] = [1 if (x <= 10) else 0 for x in test_values['geo_level_1_id']]

test_values['geo_level_1_id_10_20'] = [1 if ((x <= 20) & (x > 10))  else 0 \
                                        for x in test_values['geo_level_1_id']]

test_values['geo_level_1_id_higher_than_20'] = [1 if (x > 20) else 0 for x in test_values['geo_level_1_id']]

In [9]:
#Geo Level Id 2. 

train_values.geo_level_2_id.describe() 

train_values['geo_level_2_id_less_than_500'] = [1 if (x <= 500) else 0 for x in train_values['geo_level_2_id']]

train_values['geo_level_2_id_500_1000'] = [1 if ((x <= 1000) & (x > 500)) \
                                           else 0 for x in train_values['geo_level_2_id']]

train_values['geo_level_2_id_higher_than_1000'] = [1 if (x > 1000) else 0 for x in train_values['geo_level_2_id']]


#Hacemos lo mismo para el set de Test.


test_values['geo_level_2_id_less_than_500'] = [1 if (x <= 500) else 0 for x in test_values['geo_level_2_id']]

test_values['geo_level_2_id_500_1000'] = [1 if ((x <= 1000) & (x > 500)) \
                                           else 0 for x in test_values['geo_level_2_id']]

test_values['geo_level_2_id_higher_than_1000'] = [1 if (x > 1000) else 0 for x in test_values['geo_level_2_id']]

In [10]:
#Geo Level Id 3. 

train_values.geo_level_3_id.describe() 

train_values['geo_level_3_id_less_than_4000'] = [1 if (x <= 4000) else 0 for x in train_values['geo_level_3_id']]

train_values['geo_level_3_id_4000_8000'] = [1 if ((x <= 8000) & (x > 4000))  \
                                            else 0 for x in train_values['geo_level_3_id']]

train_values['geo_level_3_id_higher_than_8000'] = [1 if (x > 8000) else 0 for x in train_values['geo_level_3_id']]



#Hacemos lo mismo para el set de Test.


test_values['geo_level_3_id_less_than_4000'] = [1 if (x <= 4000) else 0 for x in test_values['geo_level_3_id']]

test_values['geo_level_3_id_4000_8000'] = [1 if ((x <= 8000) & (x > 4000))  \
                                            else 0 for x in test_values['geo_level_3_id']]

test_values['geo_level_3_id_higher_than_8000'] = [1 if (x > 8000) else 0 for x in test_values['geo_level_3_id']]



In [11]:
#Count Floors Pre Eq.

count_floors_pre_eq_mean = train_values['count_floors_pre_eq'].mean()

train_values['count_floor_pre_eq_higher_than_mean'] = [1 if\
                    (x > count_floors_pre_eq_mean) else 0 for x in train_values['count_floors_pre_eq']] 



#Hacemos lo mismo para el set de Test.


count_floors_pre_eq_mean = test_values['count_floors_pre_eq'].mean()

test_values['count_floor_pre_eq_higher_than_mean'] = [1 if\
                    (x > count_floors_pre_eq_mean) else 0 for x in test_values['count_floors_pre_eq']] 

In [12]:
#Age. 

train_values.age.describe()

train_values['age_less_than_20'] = [1 if (x <= 20) else 0 for x in train_values['age']]

train_values['age_20_50'] = [1 if ((x <= 50) & (x > 20))  else 0 for x in train_values['age']]

train_values['age_higher_than_50'] = [1 if (x > 50) else 0 for x in train_values['age']]



#Hacemos lo mismo para el set de Test.


test_values['age_less_than_20'] = [1 if (x <= 20) else 0 for x in test_values['age']]

test_values['age_20_50'] = [1 if ((x <= 50) & (x > 20))  else 0 for x in test_values['age']]

test_values['age_higher_than_50'] = [1 if (x > 50) else 0 for x in test_values['age']]

In [13]:
#Area and Height Percentage. Volume Percentage. 

train_values['volume_percetage'] = (train_values['area_percentage'] * train_values['height_percentage'])



#Hacemos lo mismo para el set de Test.


test_values['volume_percetage'] = (test_values['area_percentage'] * test_values['height_percentage'])

In [14]:
#Position. 

train_values = train_values.drop('position',1)


#Hacemos lo mismo para el set de Test.



test_values = test_values.drop('position',1)

In [15]:
#Materiales.

train_values['weak_superstructure'] = np.where( ( (train_values.has_superstructure_mud_mortar_stone == 1) |  \
                                       (train_values.has_superstructure_adobe_mud == 1) \
                                           | (train_values.has_superstructure_stone_flag == 1)  \
                                           | (train_values.has_superstructure_mud_mortar_brick == 1)   \
                                            | (train_values.has_superstructure_timber == 1)   \
                                            | (train_values.has_superstructure_bamboo == 1)    ), 1, 0)

train_values['strong_superstructure'] = np.where( ( (train_values.has_superstructure_cement_mortar_brick == 1) | \
                                       (train_values.has_superstructure_rc_non_engineered == 1) \
                                           | (train_values.has_superstructure_rc_engineered == 1)  ), 1, 0)


#Hacemos lo mismo para el set de Test.


test_values['weak_superstructure'] = np.where( ( (test_values.has_superstructure_mud_mortar_stone == 1) |  \
                                       (test_values.has_superstructure_adobe_mud == 1) \
                                           | (test_values.has_superstructure_stone_flag == 1)  \
                                           | (test_values.has_superstructure_mud_mortar_brick == 1)   \
                                            | (test_values.has_superstructure_timber == 1)   \
                                            | (test_values.has_superstructure_bamboo == 1)    ), 1, 0)

test_values['strong_superstructure'] = np.where( ( (test_values.has_superstructure_cement_mortar_brick == 1) | \
                                       (test_values.has_superstructure_rc_non_engineered == 1) \
                                           | (test_values.has_superstructure_rc_engineered == 1)  ), 1, 0)

In [16]:
#Count_families.

train_values['has_family'] = [1 if (x >= 1) else 0 for x in train_values['count_families']] 


#Hacemos lo mismo para el set de Test.

test_values['has_family'] = [1 if (x >= 1) else 0 for x in test_values['count_families']]

In [17]:
#Usos secundarios.

train_values['cantidad_usos_secundarios'] = train_values['has_secondary_use_agriculture'] + \
                                             train_values['has_secondary_use_hotel'] + \
                                              train_values['has_secondary_use_rental'] + \
                                              train_values['has_secondary_use_institution'] + \
                                             train_values['has_secondary_use_school'] + \
                                                train_values['has_secondary_use_industry'] + \
                                             train_values['has_secondary_use_health_post'] + \
                                                train_values['has_secondary_use_gov_office'] + \
                                                train_values['has_secondary_use_use_police'] + \
                                                train_values['has_secondary_use_other']


#Hacemos lo mismo para el set de Test.


test_values['cantidad_usos_secundarios'] = test_values['has_secondary_use_agriculture'] + \
                                             test_values['has_secondary_use_hotel'] + \
                                              test_values['has_secondary_use_rental'] + \
                                              test_values['has_secondary_use_institution'] + \
                                             test_values['has_secondary_use_school'] + \
                                                test_values['has_secondary_use_industry'] + \
                                             test_values['has_secondary_use_health_post'] + \
                                                test_values['has_secondary_use_gov_office'] + \
                                                test_values['has_secondary_use_use_police'] + \
                                                test_values['has_secondary_use_other']



# Variables Categóricas. 

In [18]:
#Land Surface Condition.



train_values.land_surface_condition.value_counts()

#Revisamos los posibles valores de esta columna. 

#Creamos entonces dos columnas mas para poder encodear en binario esta variable categorica.  


train_values['land_surface_condition_2'] = np.where( ( train_values.land_surface_condition == 'n' ), 1, 0)
train_values['land_surface_condition_1'] = np.where( ( train_values.land_surface_condition == 'o' ), 1, 0)

train_values = train_values.drop('land_surface_condition',1)



#Hacemos lo mismo para el set de Test.


test_values['land_surface_condition_2'] = np.where( ( test_values.land_surface_condition == 'n' ), 1, 0)
test_values['land_surface_condition_1'] = np.where( ( test_values.land_surface_condition == 'o' ), 1, 0)

test_values = test_values.drop('land_surface_condition',1)

In [19]:
#Foundation Type. 

train_values.foundation_type.value_counts()

#Revisamos los posibles valores de esta columna.

#Creamos entonces tres columnas mas para poder encodear en binario esta variable categorica.

train_values['foundation_type_1'] = np.where( ( train_values.foundation_type == 'h' ), 1, 0)

train_values['foundation_type_2'] = np.where( ( (train_values.foundation_type == 'u') | \
                                                     (train_values.foundation_type == 'i') ), 1, 0)

train_values['foundation_type_3'] = np.where( ( (train_values.foundation_type == 'w') | \
                                                     (train_values.foundation_type == 'i') ), 1, 0)

train_values = train_values.drop('foundation_type',1)

#Hacemos lo mismo para el set de Test.

test_values['foundation_type_1'] = np.where( ( test_values.foundation_type == 'h' ), 1, 0)

test_values['foundation_type_2'] = np.where( ( (test_values.foundation_type == 'u') | \
                                                     (test_values.foundation_type == 'i') ), 1, 0)

test_values['foundation_type_3'] = np.where( ( (test_values.foundation_type == 'w') | \
                                                     (test_values.foundation_type == 'i') ), 1, 0)

test_values = test_values.drop('foundation_type',1)

In [20]:
#Roof Type.

train_values.roof_type.value_counts()

#Revisamos los posibles valores de esta columna.

#Creamos entonces dos columnas mas para poder encodear en binario esta variable categorica.

train_values['roof_type_1'] = np.where( ( train_values.roof_type == 'x' ), 1, 0)
train_values['roof_type_2'] = np.where( ( train_values.roof_type == 'q' ), 1, 0)

train_values = train_values.drop('roof_type',1)

#Hacemos lo mismo para el set de Test.

test_values['roof_type_1'] = np.where( ( test_values.roof_type == 'x' ), 1, 0)
test_values['roof_type_2'] = np.where( ( test_values.roof_type == 'q' ), 1, 0)

test_values = test_values.drop('roof_type',1)

In [21]:
# Ground Floor Type.

train_values.ground_floor_type.value_counts()

#Revisamos los posibles valores de esta columna.

#Creamos entonces tres columnas mas para poder encodear en binario esta variable categorica.

train_values['ground_floor_type_1'] = np.where( ( train_values.ground_floor_type == 'm' ), 1, 0)

train_values['ground_floor_type_2'] = np.where( ( (train_values.ground_floor_type == 'v') | \
                                                     (train_values.ground_floor_type == 'z') ), 1, 0)

train_values['ground_floor_type_3'] = np.where( ( (train_values.ground_floor_type == 'x') | \
                                                     (train_values.ground_floor_type == 'z') ), 1, 0)

train_values = train_values.drop('ground_floor_type',1)



#Hacemos lo mismo para el set de Test.

test_values['ground_floor_type_1'] = np.where( ( test_values.ground_floor_type == 'm' ), 1, 0)

test_values['ground_floor_type_2'] = np.where( ( (test_values.ground_floor_type == 'v') | \
                                                     (test_values.ground_floor_type == 'z') ), 1, 0)

test_values['ground_floor_type_3'] = np.where( ( (test_values.ground_floor_type == 'x') | \
                                                     (test_values.ground_floor_type == 'z') ), 1, 0)

test_values = test_values.drop('ground_floor_type',1)

In [22]:
#Other Floor Type. 

train_values.other_floor_type.value_counts()

#Revisamos los posibles valores de esta columna.

#Creamos entonces dos columnas mas para poder encodear en binario esta variable categorica.

train_values['other_floor_type_1'] = np.where( ( (train_values.other_floor_type == 'j') | \
                                                     (train_values.other_floor_type == 's') ), 1, 0)

train_values['other_floor_type_2'] = np.where( ( (train_values.other_floor_type == 'x') | \
                                                     (train_values.other_floor_type == 's') ), 1, 0)

train_values = train_values.drop('other_floor_type',1)

#Hacemos lo mismo para el set de Test.

test_values['other_floor_type_1'] = np.where( ( (test_values.other_floor_type == 'j') | \
                                                     (test_values.other_floor_type == 's') ), 1, 0)

test_values['other_floor_type_2'] = np.where( ( (test_values.other_floor_type == 'x') | \
                                                     (test_values.other_floor_type == 's') ), 1, 0)

test_values = test_values.drop('other_floor_type',1)

In [23]:
#Plan Configuration

train_values.plan_configuration.value_counts()

#Revisamos los posibles valores de esta columna.

#Creamos entonces cuatro columnas mas para poder encodear en binario esta variable categorica.

train_values['plan_configuration_1'] = np.where( ( (train_values.plan_configuration == 'n') | \
                                                     (train_values.plan_configuration == 'f') ), 1, 0)

train_values['plan_configuration_2'] = np.where( ( (train_values.plan_configuration == 'c') | \
                                                     (train_values.plan_configuration == 'a') | \
                                                     (train_values.plan_configuration == 'o') | \
                                                     (train_values.plan_configuration == 'm') ), 1, 0)

train_values['plan_configuration_3'] = np.where( ( (train_values.plan_configuration == 'u') | \
                                                     (train_values.plan_configuration == 's') | \
                                                     (train_values.plan_configuration == 'o') | \
                                                     (train_values.plan_configuration == 'm') ), 1, 0)

train_values['plan_configuration_4'] = np.where( ( (train_values.plan_configuration == 'q') | \
                                                     (train_values.plan_configuration == 's') | \
                                                     (train_values.plan_configuration == 'a') | \
                                                     (train_values.plan_configuration == 'm') | \
                                                     (train_values.plan_configuration == 'f') ), 1, 0)

train_values = train_values.drop('plan_configuration',1)

#Hacemos lo mismo para el set de Test.

test_values['plan_configuration_1'] = np.where( ( (test_values.plan_configuration == 'n') | \
                                                     (test_values.plan_configuration == 'f') ), 1, 0)

test_values['plan_configuration_2'] = np.where( ( (test_values.plan_configuration == 'c') | \
                                                     (test_values.plan_configuration == 'a') | \
                                                     (test_values.plan_configuration == 'o') | \
                                                     (test_values.plan_configuration == 'm') ), 1, 0)

test_values['plan_configuration_3'] = np.where( ( (test_values.plan_configuration == 'u') | \
                                                     (test_values.plan_configuration == 's') | \
                                                     (test_values.plan_configuration == 'o') | \
                                                     (test_values.plan_configuration == 'm') ), 1, 0)

test_values['plan_configuration_4'] = np.where( ( (test_values.plan_configuration == 'q') | \
                                                     (test_values.plan_configuration == 's') | \
                                                     (test_values.plan_configuration == 'a') | \
                                                     (test_values.plan_configuration == 'm') | \
                                                     (test_values.plan_configuration == 'f') ), 1, 0)

test_values = test_values.drop('plan_configuration',1)


In [24]:
#Legal Ownership Status 

train_values.legal_ownership_status.value_counts()

#Revisamos los posibles valores de esta columna.

#Creamos entonces dos columnas mas para poder encodear en binario esta variable categorica.

train_values['legal_ownership_status_1'] = np.where( ( (train_values.legal_ownership_status == 'w') | \
                                                     (train_values.legal_ownership_status == 'r') ), 1, 0)

train_values['legal_ownership_status_2'] = np.where( ( (train_values.legal_ownership_status == 'a') | \
                                                     (train_values.legal_ownership_status == 'r') ), 1, 0)

train_values = train_values.drop('legal_ownership_status',1)

#Hacemos lo mismo para el set de Test.

test_values['legal_ownership_status_1'] = np.where( ( (test_values.legal_ownership_status == 'w') | \
                                                     (test_values.legal_ownership_status == 'r') ), 1, 0)

test_values['legal_ownership_status_2'] = np.where( ( (test_values.legal_ownership_status == 'a') | \
                                                     (test_values.legal_ownership_status == 'r') ), 1, 0)

test_values = test_values.drop('legal_ownership_status',1)

# Tuning de Hiperparámetros.

In [134]:


parameters = {'depth'         : [5,7,9],
              'learning_rate' : [0.05,0.3], # Para la proxima vamos a poner solo dos de estos valores, vemos cual nos da mejor, y despues agregamos los otros 3. Hay que agregar 0.1, 0.5 y 1.  
              'l2_leaf_reg': [3,1,5,10,100],
              #'border_count' : [32,5,10,20,50,100,200],  
              'iterations'    : [200,500,750,1000,1500]
                 }

# l2_leaf_reg= Se acepta cualquier numero positivo. El tipo prueba con [3,1,5,10,100].  

# border_count Numeros de 1 a 255. El tipo prueba con [32,5,10,20,50,100,200].

#Tambien faltaria agregar el random_strength 

#Tambien faltaria agregar el bagging_temperature que acepta cualquier valor positivo. 

#Despues de estos, no hay que tunear ningun hiperparametro mas. 

model = CatBoostClassifier()

Grid_CBC = GridSearchCV(estimator=model, param_grid = parameters, cv = 2, n_jobs=-1, scoring = f1)

Grid_CBC.fit(train_values, train_labels)

print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n",Grid_CBC.best_estimator_)
print("\n The best score across ALL searched params:\n",Grid_CBC.best_score_)
print("\n The best parameters across ALL searched params:\n",Grid_CBC.best_params_)

#De esto lo mejor que dió fue depth 7, iterations 1000, learning_rate 0.3. 

#parameters = {'depth'         : [5,6,7],
#                 'learning_rate' : [0.1,0.05,0.2,0.3,0.02],
#                  'iterations'    : [100,200,500,750,1000]
#                 }

# -----------------------------------------------------------------------------

#De esto lo mejor que dió fue  iterations 1500, learning_rate 0.3. 


#parameters = {
#                 'learning_rate' : [0.3,0.4],
#                  'iterations'    : [1500,1000]
#                 }

# -----------------------------------------------------------------------------

/home/fedepenic/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


KeyboardInterrupt: 

# Probamos el mejor modelo obtenido.

In [92]:

#Este modelo nos dio como resultado en Driven Data 0.7317, aca me habia dado algo de 0,7630
#-> depth 7, iterations 500, learning_rate 0.4. 

#Este modelo nos dio como resultado en Driven Data 0.7303, aca me habia dado 0,75 clavado. 
#-> depth 6, iterations 700, learning_rate 0.250. Se redujo el Overfitting, pero tambien
# hay que considerar que aca hice el Feature Engineering. 

# Este modelo nos dio como resultado en Driven Data 0.7355, aca nos dio 0.7788.
# Fueron 7 depth, iterations 1000, learning rate 0.3 

# Este modelo nos dio como resultado en Driven Data 0.7306, aca nos dio o.8546.
#Fue 8 depth, iterations 1500, learning rate 0.5.

# Depth 7, iterations 1500, learning rate 0.3 me dio aca 0.7957, todavia no lo probe en Driven Data. 

#Quizas habria que probar con un numero bajo de iterations < 500.  


finalModel = CatBoostClassifier(depth = 7, iterations = 1500, learning_rate = 0.3)

finalModel.fit(train_values, train_labels)
preds = finalModel.predict(train_values)

0:	learn: 0.9448652	total: 64.3ms	remaining: 1m 36s
1:	learn: 0.8671157	total: 128ms	remaining: 1m 36s
2:	learn: 0.8183689	total: 197ms	remaining: 1m 38s
3:	learn: 0.7885904	total: 259ms	remaining: 1m 36s
4:	learn: 0.7712402	total: 335ms	remaining: 1m 40s
5:	learn: 0.7535165	total: 400ms	remaining: 1m 39s
6:	learn: 0.7436097	total: 467ms	remaining: 1m 39s
7:	learn: 0.7352040	total: 530ms	remaining: 1m 38s
8:	learn: 0.7289063	total: 609ms	remaining: 1m 40s
9:	learn: 0.7238986	total: 679ms	remaining: 1m 41s
10:	learn: 0.7181051	total: 745ms	remaining: 1m 40s
11:	learn: 0.7153319	total: 810ms	remaining: 1m 40s
12:	learn: 0.7115168	total: 906ms	remaining: 1m 43s
13:	learn: 0.7098247	total: 986ms	remaining: 1m 44s
14:	learn: 0.7066174	total: 1.06s	remaining: 1m 45s
15:	learn: 0.7021500	total: 1.14s	remaining: 1m 45s
16:	learn: 0.7001791	total: 1.2s	remaining: 1m 44s
17:	learn: 0.6984791	total: 1.27s	remaining: 1m 44s
18:	learn: 0.6969678	total: 1.35s	remaining: 1m 45s
19:	learn: 0.6948703	t

158:	learn: 0.6140408	total: 11.8s	remaining: 1m 39s
159:	learn: 0.6138338	total: 11.9s	remaining: 1m 39s
160:	learn: 0.6136539	total: 11.9s	remaining: 1m 39s
161:	learn: 0.6135005	total: 12s	remaining: 1m 38s
162:	learn: 0.6132476	total: 12.1s	remaining: 1m 38s
163:	learn: 0.6129877	total: 12.1s	remaining: 1m 38s
164:	learn: 0.6127430	total: 12.2s	remaining: 1m 38s
165:	learn: 0.6125970	total: 12.3s	remaining: 1m 38s
166:	learn: 0.6123713	total: 12.3s	remaining: 1m 38s
167:	learn: 0.6119247	total: 12.4s	remaining: 1m 38s
168:	learn: 0.6116892	total: 12.5s	remaining: 1m 38s
169:	learn: 0.6115300	total: 12.5s	remaining: 1m 38s
170:	learn: 0.6113329	total: 12.6s	remaining: 1m 37s
171:	learn: 0.6109749	total: 12.7s	remaining: 1m 37s
172:	learn: 0.6107838	total: 12.7s	remaining: 1m 37s
173:	learn: 0.6103528	total: 12.8s	remaining: 1m 37s
174:	learn: 0.6101774	total: 12.9s	remaining: 1m 37s
175:	learn: 0.6100069	total: 13s	remaining: 1m 37s
176:	learn: 0.6096351	total: 13s	remaining: 1m 37s

314:	learn: 0.5843900	total: 23.1s	remaining: 1m 27s
315:	learn: 0.5841384	total: 23.2s	remaining: 1m 27s
316:	learn: 0.5840077	total: 23.3s	remaining: 1m 26s
317:	learn: 0.5839294	total: 23.4s	remaining: 1m 26s
318:	learn: 0.5838086	total: 23.4s	remaining: 1m 26s
319:	learn: 0.5836779	total: 23.5s	remaining: 1m 26s
320:	learn: 0.5835472	total: 23.6s	remaining: 1m 26s
321:	learn: 0.5834465	total: 23.6s	remaining: 1m 26s
322:	learn: 0.5832406	total: 23.7s	remaining: 1m 26s
323:	learn: 0.5830173	total: 23.8s	remaining: 1m 26s
324:	learn: 0.5828799	total: 23.9s	remaining: 1m 26s
325:	learn: 0.5826300	total: 23.9s	remaining: 1m 26s
326:	learn: 0.5825909	total: 24s	remaining: 1m 26s
327:	learn: 0.5824201	total: 24.1s	remaining: 1m 26s
328:	learn: 0.5822378	total: 24.1s	remaining: 1m 25s
329:	learn: 0.5820905	total: 24.2s	remaining: 1m 25s
330:	learn: 0.5819923	total: 24.3s	remaining: 1m 25s
331:	learn: 0.5818350	total: 24.3s	remaining: 1m 25s
332:	learn: 0.5817374	total: 24.4s	remaining: 1m

471:	learn: 0.5637473	total: 38.3s	remaining: 1m 23s
472:	learn: 0.5636362	total: 38.4s	remaining: 1m 23s
473:	learn: 0.5635853	total: 38.4s	remaining: 1m 23s
474:	learn: 0.5635338	total: 38.5s	remaining: 1m 23s
475:	learn: 0.5634864	total: 38.6s	remaining: 1m 22s
476:	learn: 0.5633625	total: 38.6s	remaining: 1m 22s
477:	learn: 0.5632912	total: 38.7s	remaining: 1m 22s
478:	learn: 0.5631540	total: 38.8s	remaining: 1m 22s
479:	learn: 0.5630004	total: 38.9s	remaining: 1m 22s
480:	learn: 0.5629367	total: 38.9s	remaining: 1m 22s
481:	learn: 0.5628381	total: 39s	remaining: 1m 22s
482:	learn: 0.5627192	total: 39.1s	remaining: 1m 22s
483:	learn: 0.5625720	total: 39.2s	remaining: 1m 22s
484:	learn: 0.5624227	total: 39.2s	remaining: 1m 22s
485:	learn: 0.5623218	total: 39.3s	remaining: 1m 22s
486:	learn: 0.5621830	total: 39.4s	remaining: 1m 21s
487:	learn: 0.5620509	total: 39.5s	remaining: 1m 21s
488:	learn: 0.5618974	total: 39.6s	remaining: 1m 21s
489:	learn: 0.5617254	total: 39.6s	remaining: 1m

627:	learn: 0.5472212	total: 51.1s	remaining: 1m 11s
628:	learn: 0.5470375	total: 51.2s	remaining: 1m 10s
629:	learn: 0.5469502	total: 51.3s	remaining: 1m 10s
630:	learn: 0.5468745	total: 51.4s	remaining: 1m 10s
631:	learn: 0.5467766	total: 51.4s	remaining: 1m 10s
632:	learn: 0.5467082	total: 51.5s	remaining: 1m 10s
633:	learn: 0.5466334	total: 51.6s	remaining: 1m 10s
634:	learn: 0.5465457	total: 51.7s	remaining: 1m 10s
635:	learn: 0.5464240	total: 51.8s	remaining: 1m 10s
636:	learn: 0.5464001	total: 51.8s	remaining: 1m 10s
637:	learn: 0.5463091	total: 51.9s	remaining: 1m 10s
638:	learn: 0.5462249	total: 52s	remaining: 1m 10s
639:	learn: 0.5460456	total: 52s	remaining: 1m 9s
640:	learn: 0.5458963	total: 52.1s	remaining: 1m 9s
641:	learn: 0.5458085	total: 52.2s	remaining: 1m 9s
642:	learn: 0.5456938	total: 52.3s	remaining: 1m 9s
643:	learn: 0.5456084	total: 52.3s	remaining: 1m 9s
644:	learn: 0.5455340	total: 52.4s	remaining: 1m 9s
645:	learn: 0.5454254	total: 52.5s	remaining: 1m 9s
646:

789:	learn: 0.5321979	total: 1m 4s	remaining: 58.2s
790:	learn: 0.5321187	total: 1m 4s	remaining: 58.1s
791:	learn: 0.5320380	total: 1m 4s	remaining: 58s
792:	learn: 0.5319358	total: 1m 5s	remaining: 58s
793:	learn: 0.5318130	total: 1m 5s	remaining: 57.9s
794:	learn: 0.5317679	total: 1m 5s	remaining: 57.8s
795:	learn: 0.5316557	total: 1m 5s	remaining: 57.7s
796:	learn: 0.5315383	total: 1m 5s	remaining: 57.7s
797:	learn: 0.5314803	total: 1m 5s	remaining: 57.6s
798:	learn: 0.5314007	total: 1m 5s	remaining: 57.5s
799:	learn: 0.5312459	total: 1m 5s	remaining: 57.4s
800:	learn: 0.5311521	total: 1m 5s	remaining: 57.3s
801:	learn: 0.5310904	total: 1m 5s	remaining: 57.2s
802:	learn: 0.5309940	total: 1m 5s	remaining: 57.1s
803:	learn: 0.5308634	total: 1m 5s	remaining: 57s
804:	learn: 0.5307934	total: 1m 5s	remaining: 57s
805:	learn: 0.5307255	total: 1m 6s	remaining: 56.9s
806:	learn: 0.5306178	total: 1m 6s	remaining: 56.8s
807:	learn: 0.5305623	total: 1m 6s	remaining: 56.7s
808:	learn: 0.530515

948:	learn: 0.5190597	total: 1m 20s	remaining: 46.4s
949:	learn: 0.5189295	total: 1m 20s	remaining: 46.4s
950:	learn: 0.5188343	total: 1m 20s	remaining: 46.3s
951:	learn: 0.5187619	total: 1m 20s	remaining: 46.2s
952:	learn: 0.5186831	total: 1m 20s	remaining: 46.1s
953:	learn: 0.5186235	total: 1m 20s	remaining: 46s
954:	learn: 0.5185449	total: 1m 20s	remaining: 45.9s
955:	learn: 0.5184859	total: 1m 20s	remaining: 45.8s
956:	learn: 0.5184396	total: 1m 20s	remaining: 45.7s
957:	learn: 0.5183832	total: 1m 20s	remaining: 45.6s
958:	learn: 0.5182983	total: 1m 20s	remaining: 45.6s
959:	learn: 0.5181813	total: 1m 20s	remaining: 45.5s
960:	learn: 0.5181002	total: 1m 20s	remaining: 45.4s
961:	learn: 0.5180283	total: 1m 21s	remaining: 45.3s
962:	learn: 0.5179648	total: 1m 21s	remaining: 45.2s
963:	learn: 0.5178889	total: 1m 21s	remaining: 45.1s
964:	learn: 0.5178169	total: 1m 21s	remaining: 45s
965:	learn: 0.5177456	total: 1m 21s	remaining: 44.9s
966:	learn: 0.5176922	total: 1m 21s	remaining: 44.

1103:	learn: 0.5077383	total: 1m 32s	remaining: 33.1s
1104:	learn: 0.5076786	total: 1m 32s	remaining: 33s
1105:	learn: 0.5076335	total: 1m 32s	remaining: 32.9s
1106:	learn: 0.5075954	total: 1m 32s	remaining: 32.8s
1107:	learn: 0.5075378	total: 1m 32s	remaining: 32.7s
1108:	learn: 0.5074732	total: 1m 32s	remaining: 32.6s
1109:	learn: 0.5074244	total: 1m 32s	remaining: 32.6s
1110:	learn: 0.5073495	total: 1m 32s	remaining: 32.5s
1111:	learn: 0.5072837	total: 1m 32s	remaining: 32.4s
1112:	learn: 0.5072362	total: 1m 32s	remaining: 32.3s
1113:	learn: 0.5071924	total: 1m 32s	remaining: 32.2s
1114:	learn: 0.5071192	total: 1m 33s	remaining: 32.1s
1115:	learn: 0.5070366	total: 1m 33s	remaining: 32s
1116:	learn: 0.5069512	total: 1m 33s	remaining: 31.9s
1117:	learn: 0.5068556	total: 1m 33s	remaining: 31.9s
1118:	learn: 0.5067621	total: 1m 33s	remaining: 31.8s
1119:	learn: 0.5067172	total: 1m 33s	remaining: 31.7s
1120:	learn: 0.5066664	total: 1m 33s	remaining: 31.6s
1121:	learn: 0.5066324	total: 1m

1256:	learn: 0.4973161	total: 1m 43s	remaining: 20s
1257:	learn: 0.4972542	total: 1m 43s	remaining: 20s
1258:	learn: 0.4971861	total: 1m 43s	remaining: 19.9s
1259:	learn: 0.4971247	total: 1m 43s	remaining: 19.8s
1260:	learn: 0.4970548	total: 1m 43s	remaining: 19.7s
1261:	learn: 0.4970251	total: 1m 44s	remaining: 19.6s
1262:	learn: 0.4969450	total: 1m 44s	remaining: 19.5s
1263:	learn: 0.4968869	total: 1m 44s	remaining: 19.5s
1264:	learn: 0.4968110	total: 1m 44s	remaining: 19.4s
1265:	learn: 0.4967217	total: 1m 44s	remaining: 19.3s
1266:	learn: 0.4966587	total: 1m 44s	remaining: 19.2s
1267:	learn: 0.4965929	total: 1m 44s	remaining: 19.1s
1268:	learn: 0.4965601	total: 1m 44s	remaining: 19.1s
1269:	learn: 0.4965107	total: 1m 44s	remaining: 19s
1270:	learn: 0.4964731	total: 1m 45s	remaining: 18.9s
1271:	learn: 0.4963746	total: 1m 45s	remaining: 18.8s
1272:	learn: 0.4963304	total: 1m 45s	remaining: 18.8s
1273:	learn: 0.4962514	total: 1m 45s	remaining: 18.7s
1274:	learn: 0.4961734	total: 1m 4

1411:	learn: 0.4874301	total: 1m 57s	remaining: 7.3s
1412:	learn: 0.4873293	total: 1m 57s	remaining: 7.21s
1413:	learn: 0.4872819	total: 1m 57s	remaining: 7.13s
1414:	learn: 0.4872334	total: 1m 57s	remaining: 7.05s
1415:	learn: 0.4871954	total: 1m 57s	remaining: 6.96s
1416:	learn: 0.4871194	total: 1m 57s	remaining: 6.88s
1417:	learn: 0.4870732	total: 1m 57s	remaining: 6.8s
1418:	learn: 0.4869963	total: 1m 57s	remaining: 6.71s
1419:	learn: 0.4869085	total: 1m 57s	remaining: 6.63s
1420:	learn: 0.4868256	total: 1m 57s	remaining: 6.55s
1421:	learn: 0.4867426	total: 1m 57s	remaining: 6.46s
1422:	learn: 0.4866930	total: 1m 57s	remaining: 6.38s
1423:	learn: 0.4866415	total: 1m 58s	remaining: 6.3s
1424:	learn: 0.4865934	total: 1m 58s	remaining: 6.22s
1425:	learn: 0.4865651	total: 1m 58s	remaining: 6.14s
1426:	learn: 0.4864924	total: 1m 58s	remaining: 6.05s
1427:	learn: 0.4864609	total: 1m 58s	remaining: 5.97s
1428:	learn: 0.4864185	total: 1m 58s	remaining: 5.89s
1429:	learn: 0.4863591	total: 1

# Verificamos el Score del mejor modelo para el set Train. 

In [93]:
preds=[int(x+0.5) for x in preds]

f1_score(train_labels, preds, average='micro')

0.7957835925418552

# Hacemos las predicciones para el set de Test. 

In [47]:
prediccionesFinales = finalModel.predict(test_values)

# Submission de Resultados.

In [48]:
my_submission = pd.DataFrame(data=prediccionesFinales,
                             columns=submission_format.columns,
                             index=submission_format.index)

my_submission.damage_grade = my_submission.damage_grade.round()

my_submission.damage_grade = my_submission.damage_grade.astype(int)

my_submission.to_csv('submissionCatboost.csv')